## Exploring Few-Shot Prompt Engineering

[Few-Shot Prompting](https://www.promptingguide.ai/techniques/fewshot) is a way of guiding a natural language processing model to respond through examples.

The technique's basis comes from the tendency of AI models to perform Similarity Searches, a complex topic which I covered lightly in [Faiss Vectors](https://github.com/ginobaltazar7/66daysofdata/tree/master/FaissVectorDB). In case you missed it, here is a [link to a good starter series](https://towardsdatascience.com/similarity-search-knn-inverted-file-index-7cab80cc0e79) for those curious.

In [2]:
!pip install langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 19.8 MB/s eta 0:00:00


In [12]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00


In [23]:
from langchain.chat_models import ChatOpenAI

In [24]:
from langchain.chains import ConversationChain

chat_model = ChatOpenAI(openai_api_key='enter-your-own-api-key-here')
conversation_chain = ConversationChain(
    llm=chat_model
)

conversation_chain.prompt.template

'The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'

In [25]:
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)

examples = [
    {"input": "12+12", "output": "24"},
    {"input": "12+13", "output": "25"},
    {"input": "15+16", "output": "31"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: 12+12
AI: 24
Human: 12+13
AI: 25
Human: 15+16
AI: 31


In [26]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Are you good at math"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(final_prompt.format(input='What is 13+13?'))

System: Are you good at math
Human: 12+12
AI: 24
Human: 12+13
AI: 25
Human: 15+16
AI: 31
Human: What is 13+13?


In [28]:
# Feed to an LLM
from langchain.chains import LLMChain

chain = LLMChain(
    llm=chat_model,
    prompt=final_prompt,
    verbose=True
)

chain.run("What is 13+13?")



> Entering new LLMChain chain...
Prompt after formatting:
System: Are you good at math
Human: 12+12
AI: 24
Human: 12+13
AI: 25
Human: 15+16
AI: 31
Human: What is 13+13?

> Finished chain.


'26'

In [30]:
# try language translation

examples = [
    {"input": "car", "output": "voiture"},
    {"input": "house", "output": "maison"},
    {"input": "backpack", "output": "sac-a-dos"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: car
AI: voiture
Human: house
AI: maison
Human: backpack
AI: sac-a-dos


In [31]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Are you good at French"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(final_prompt.format(input='What does child translate to in French?'))

System: Are you good at French
Human: car
AI: voiture
Human: house
AI: maison
Human: backpack
AI: sac-a-dos
Human: What does child translate to in French?


In [32]:
# Feed to an LLM
from langchain.chains import LLMChain

chain = LLMChain(
    llm=chat_model,
    prompt=final_prompt,
    verbose=True
)

chain.run("What does child translate to in French?")



> Entering new LLMChain chain...
Prompt after formatting:
System: Are you good at French
Human: car
AI: voiture
Human: house
AI: maison
Human: backpack
AI: sac-a-dos
Human: What does child translate to in French?

> Finished chain.


'Enfant'